# MedMask 库优越性展示

本示例展示了 MedMask 库在医学图像分割掩膜处理方面的优越性。

## 核心优势

1. **文件管理** - 117个分散文件 → 1个归档文件
2. **存储优化** - Zstandard压缩，显著减少空间占用
3. **语义化** - 内置名称到标签的双向映射
4. **性能提升** - 优化的I/O和批量操作
5. **API简洁** - 一行代码完成复杂操作
6. **重叠掩膜** - 支持多粒度的掩膜组合

## 数据

使用 TotalSegmentator 的117个器官分割结果，展示两种处理方式的对比。


In [9]:
import os
import time
import numpy as np
import nibabel as nib
from pathlib import Path

# 导入 MedMask 库
import sys
sys.path.append('../')
from medmask import SegmentationMask, MaskArchive
from spacetransformer import Space

# 数据准备
data_dir = Path("s0000")
all_files = sorted(data_dir.glob("*.nii.gz"))
first_file = nib.load(all_files[0])
space = Space.from_nifty(first_file).reverse_axis_order()

print(f"数据: {len(all_files)} 个器官文件")
print(f"图像: {first_file.get_fdata().shape} @ {space.spacing[0]:.1f}mm")

# 计算传统方式总大小
traditional_size = sum(f.stat().st_size for f in all_files)
print(f"传统总大小: {traditional_size / (1024*1024):.2f} MB")


数据: 117 个器官文件
图像: (294, 192, 179) @ 1.5mm
传统总大小: 5.12 MB


## 两种处理方式对比

### 方式一：独立语义掩膜
每个器官作为独立的 `SegmentationMask`，便于单独访问和修改，大小较大，适用于会有重叠的掩码管理

### 方式二：合并多标签掩膜  
所有器官合并到一个多标签掩膜中，使用 `add_label()` 统一管理，大小更小，适用于没有重叠的掩码管理


In [10]:
# 方式一：独立语义掩膜
print("方式一：创建独立掩膜归档...")

read_time = time.time()
all_mask_data = {}
for i, organ_file in enumerate(all_files):
    nii = nib.load(organ_file)
    mask_data = np.array(nii.dataobj)
    all_mask_data[organ_file] = mask_data
read_time = time.time() - read_time

start_time = time.time()
archive1 = MaskArchive("organs_individual.maska", mode="w", space=space)
for i, organ_file in enumerate(all_files):
    mask_data = all_mask_data[organ_file]
    organ_name = organ_file.stem.replace('.nii', '')
    
    mask = SegmentationMask(
        mask_array=mask_data.astype(np.uint8),
        mapping={organ_name: 1},
        space=space
    )
    archive1.add_mask(mask, organ_name)

time1 = time.time() - start_time
size1 = Path("organs_individual.maska").stat().st_size

# 方式二：合并多标签掩膜
print("方式二：创建合并掩膜归档...")
start_time = time.time()

archive2 = MaskArchive("organs_combined.maska", mode="w", space=space)
combined_mask = SegmentationMask.lazy_init(bit_depth=8, space=space)
all_keys = []
for i, organ_file in enumerate(all_files):
    nii = nib.load(organ_file)
    mask_data = nii.get_fdata() > 0
    organ_name = organ_file.stem.replace('.nii', '')
    all_keys.append(organ_name)
    combined_mask.add_label(mask_data, i + 1, organ_name)

archive2.add_mask(combined_mask, "all_organs_combined")

time2 = time.time() - start_time
size2 = Path("organs_combined.maska").stat().st_size

print(f"\n创建时间: 方式一 {time1:.1f}s, 方式二 {time2:.1f}s")
print(f"文件大小: 方式一 {size1/1024:.0f}KB, 方式二 {size2/1024:.0f}KB")
print(f"压缩比: 方式一 {traditional_size/size1:.1f}:1, 方式二 {traditional_size/size2:.1f}:1")


方式一：创建独立掩膜归档...


/tmp/ipykernel_1147402/816815646.py:8: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  mask_data = np.array(nii.dataobj)


方式二：创建合并掩膜归档...

创建时间: 方式一 7.3s, 方式二 6.3s
文件大小: 方式一 158KB, 方式二 92KB
压缩比: 方式一 33.1:1, 方式二 56.7:1


In [11]:
# 读取性能测试
test_organs = all_keys

# 测试传统方式
traditional_read_time = read_time

# 测试方式1 - 独立掩膜
start = time.time()
archive1_read = MaskArchive("organs_individual.maska", mode="r")
for organ in test_organs:
    mask = archive1_read.load_mask(organ)
    data = mask.get_all_masks()
method1_read_time = time.time() - start

# 测试方式2 - 合并掩膜
start = time.time()
archive2_read = MaskArchive("organs_combined.maska", mode="r")
all_organs = archive2_read.load_mask("all_organs_combined")
for organ in test_organs:
    data = all_organs.get_mask_by_names(organ)
method2_read_time = time.time() - start

print(f"读取 {len(test_organs)} 个器官:")
print(f"传统方式: {traditional_read_time:.3f}s")
speedup1 = traditional_read_time / method1_read_time
print(f"方式1 (独立掩膜): {method1_read_time:.3f}s (加速 {speedup1:.1f}x)")
speedup2 = traditional_read_time / method2_read_time
print(f"方式2 (合并掩膜): {method2_read_time:.3f}s (加速 {speedup2:.1f}x)")



读取 117 个器官:
传统方式: 1.499s
方式1 (独立掩膜): 0.121s (加速 12.3x)
方式2 (合并掩膜): 0.122s (加速 12.2x)


## 重叠掩膜优势 - 肋骨示例

展示在同一个归档中存储多个粒度的掩膜：
- 24个单独肋骨
- 左侧/右侧肋骨组合  
- 上/下肋骨组合
- 所有肋骨合并

支持从单个肋骨到全局肋骨的多层次查询。


In [12]:
# 肋骨掩膜合并示例
rib_files = sorted(data_dir.glob("rib_*.nii.gz"))
ribs_archive = MaskArchive("ribs_archive.maska", mode="w", space=space)

# 准备组合掩膜
shape = first_file.get_fdata().shape
left_ribs = np.zeros(shape, dtype=bool)
right_ribs = np.zeros(shape, dtype=bool)
all_ribs = np.zeros(shape, dtype=bool)
upper_ribs = np.zeros(shape, dtype=bool)  
lower_ribs = np.zeros(shape, dtype=bool)

print(f"处理 {len(rib_files)} 个肋骨文件...")

# 处理每个肋骨
for rib_file in rib_files:
    nii = nib.load(rib_file)
    rib_data = nii.get_fdata() > 0
    rib_name = rib_file.stem.replace('.nii', '')
    
    # 添加单独肋骨
    mask = SegmentationMask(rib_data.astype(np.uint8), {rib_name: 1}, space)
    ribs_archive.add_mask(mask, rib_name)
    
    # 合并到组合掩膜
    all_ribs |= rib_data
    if 'left' in rib_name:
        left_ribs |= rib_data
    elif 'right' in rib_name:
        right_ribs |= rib_data
    
    # 上下分组
    for i in range(1, 13):
        if f'_{i}' in rib_name:
            if i <= 6:
                upper_ribs |= rib_data
            else:
                lower_ribs |= rib_data
            break

# 添加组合掩膜
combinations = [
    ("all_ribs", all_ribs), ("left_ribs", left_ribs), ("right_ribs", right_ribs),
    ("upper_ribs", upper_ribs), ("lower_ribs", lower_ribs)
]

for name, mask_data in combinations:
    if np.any(mask_data):
        mask = SegmentationMask(mask_data.astype(np.uint8), {name: 1}, space)
        ribs_archive.add_mask(mask, name)

rib_size = Path("ribs_archive.maska").stat().st_size
print(f"肋骨归档: {len(ribs_archive.all_names())} 个掩膜, {rib_size/1024:.0f}KB")


处理 24 个肋骨文件...
肋骨归档: 27 个掩膜, 14KB


## 应用场景

**独立掩膜方式** 适合：
- 频繁访问单个器官
- 独立修改特定区域
- 模块化数据管理

**合并掩膜方式** 适合：
- 全局器官分析
- 统一标签管理
- 最大化存储效率

**重叠掩膜** 支持：
- 多粒度的解剖学查询
- 层次化的器官组织
- 灵活的区域组合分析

---

MedMask 通过先进的压缩技术、语义化设计和灵活的API，为医学图像分割掩膜提供了全面的解决方案。
